In [ ]:
# Install dependencis in your terminal
# pip install mediapipe opencv-python


In [1]:
# Import all needed packages
import mediapipe as mp
import cv2
import numpy as np
import os
import uuid
import math
from interface import *

[10]
[20]


In [2]:
# Mediapipe confg variables
mp_drawing = mp.solutions.drawing_utils # Module for rendering the landmarks on video/image
mp_hands = mp.solutions.hands # Module for work with hands - detection and tracking 

In [3]:
# Active WebCam
cap = cv2.VideoCapture(0)
img_height = cap.get(4)
img_width = cap.get(3)

with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence= 0.5) as hands:
    while cap.isOpened():
        _, frame = cap.read() # Catches the frame from video

        # Hands detections
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) # Change frame's colors into RGB scheme
        image.flags.writeable = False # Set flag to false
        results = hands.process(image)  # Detections
        image.flags.writeable = True # Set flagto true
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # Rechange colors
        
        # Draw detections on frame
        if results.multi_hand_landmarks:
            for num, hand in enumerate(results.multi_hand_landmarks):
                mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS)
            try:
                if len(coord_list) < 10:
                    coord_list.append([hand.landmark[8].x, hand.landmark[8].y])
                else:
                    coord_list.pop(0)
                    coord_list.append([hand.landmark[8].x, hand.landmark[8].y])
                print(coord_list)
            except NameError:
                coord_list = [[hand.landmark[8].x, hand.landmark[8].y]]    

            cv2.line(image, denormalization(hand, 8, img_height, img_width), denormalization(hand, 4, img_height, img_width), (255, 0, 0), 3)

        cv2.imshow('Hand Tracking', cv2.flip(image,1))  # Display frame; Flip display image as mirrored

        # Get last 8 bits from pressed key.
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release() # Release WebCam
    cv2.destroyAllWindows() 



[[0.8229549527168274, 0.3391766846179962], [0.7831282019615173, 0.23010972142219543]]
[[0.8229549527168274, 0.3391766846179962], [0.7831282019615173, 0.23010972142219543], [0.7570462226867676, 0.1816260814666748]]
[[0.8229549527168274, 0.3391766846179962], [0.7831282019615173, 0.23010972142219543], [0.7570462226867676, 0.1816260814666748], [0.7320067882537842, 0.1601943075656891]]
[[0.8229549527168274, 0.3391766846179962], [0.7831282019615173, 0.23010972142219543], [0.7570462226867676, 0.1816260814666748], [0.7320067882537842, 0.1601943075656891], [0.6241124272346497, 0.13947203755378723]]
[[0.8229549527168274, 0.3391766846179962], [0.7831282019615173, 0.23010972142219543], [0.7570462226867676, 0.1816260814666748], [0.7320067882537842, 0.1601943075656891], [0.6241124272346497, 0.13947203755378723], [0.5865439772605896, 0.15807297825813293]]
[[0.8229549527168274, 0.3391766846179962], [0.7831282019615173, 0.23010972142219543], [0.7570462226867676, 0.1816260814666748], [0.7320067882537842

In [ ]:
print(hand)